In [1]:
# 경로 설정 - 하지 않으면 기본 경로에 생성됨
!export BENTOML_HOME='~'

# 1. Model Training

In [2]:
from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)

SVC()

In [3]:
import bentoml
bentoml.sklearn.save_model("iris_clf", clf)

Model(tag="iris_clf:5uegimxtucsusaav", path="/home/wfs/bentoml/models/iris_clf/5uegimxtucsusaav/")

In [4]:
# Model List 확인
!bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:5uegimxtucsusaav  bentoml.sklearn  5.66 KiB  2022-06-24 18:35:03 
 iris_clf:nzqqd6htt6vdeaav  bentoml.sklearn  5.66 KiB  2022-06-24 18:24:21 
 iris_clf:wnhjb3htt2rfwaav  bentoml.sklearn  5.66 KiB  2022-06-24 18:19:08 
 iris_clf:vwdazmxtt2rfwaav  bentoml.sklearn  5.66 KiB  2022-06-24 18:18:58 


In [5]:
# 특정 버전 모델 가져오기
!bentoml models get iris_clf:vwdazmxtt2rfwaav

name: iris_clf                                                                  
version: vwdazmxtt2rfwaav                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.0.2                                                         
  bentoml_version: 1.0.0rc2                                                     
  python_version: 3.8.10                                                        
signatures:                 

In [6]:
# 가장 최신 버전 가져오기
!bentoml models get iris_clf:latest

name: iris_clf                                                                  
version: 5uegimxtucsusaav                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.0.2                                                         
  bentoml_version: 1.0.0rc2                                                     
  python_version: 3.8.10                                                        
signatures:                 

# 2. Model Serving - Runner

In [7]:
import bentoml

model1 = bentoml.sklearn.get('iris_clf:latest').to_runner()
model2 = bentoml.sklearn.get('iris_clf:vwdazmxtt2rfwaav').to_runner()

model1.init_local()
model2.init_local()

'Runner.init_local' is for debugging and testing only
'Runner.init_local' is for debugging and testing only


In [8]:
print(f"Model1 Predict : {model1.predict.run([[5.9, 3, 5.1, 1.8]])}")
print(f"Model2 Predict : {model2.predict.run([[5.9, 3, 5.1, 1.8]])}")

Model1 Predict : [2]
Model2 Predict : [2]


# 3. Model Serving - Service

In [9]:
%%writefile service.py

import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.predict.run(input_series)


Overwriting service.py


In [10]:
'''
BentoML 서비스 실행 - 별도의 cli나 ipynb 파일 등에서 실행

bentoml serve service.py:svc --reload

'''

'\nBentoML 서비스 실행 - 별도의 cli나 ipynb 파일 등에서 실행\n\nbentoml serve service.py:svc --reload\n\n'

In [11]:
# Python을 통해 http client requests
import requests
requests.post(
    "http://127.0.0.1:3000/classify",
    headers={"content-type": "application/json"},
    data="[[5.9, 3, 5.1, 1.8]]").text

'[2]'

In [18]:
response = requests.post(
    "http://127.0.0.1:3000/classify",
    headers={"content-type": "application/json"},
    data="[[5.9, 3, 5.1, 1.8]]")
response.json()

[2]

In [12]:
!curl -X POST -H "content-type: application/json" --data "[[5.9, 3, 5.1, 1.8]]" http://127.0.0.1:3000/classify

[2]